In [3]:
# ============================================================================
# CELL 1: Data Loading & Initial Exploration
# ============================================================================
"""
Goal: Understand the data structure and sampling requirements
- Filter US locale + Exact matches only
- Sample ~50 queries, ~500 rows total
"""

import pandas as pd
import numpy as np

# Load data
examples = pd.read_parquet("shopping_queries_dataset_examples.parquet", engine='fastparquet')
products = pd.read_parquet("shopping_queries_dataset_products.parquet", engine='fastparquet')

print(f"Raw data: {len(examples):,} examples, {len(products):,} products")
print(f"Label distribution:\n{examples['esci_label'].value_counts()}\n")

# Filter per requirements: US + Exact matches
filtered = examples[
    (examples['product_locale'] == 'us') & 
    (examples['esci_label'] == 'E')
].copy()

print(f"After filtering: {len(filtered):,} rows")
print(f"Unique queries: {filtered['query_id'].nunique():,}")
print(f"Unique products: {filtered['product_id'].nunique():,}")

# Check products per query distribution
ppq = filtered.groupby('query_id').size()
print(f"\nProducts per query: mean={ppq.mean():.1f}, median={ppq.median():.0f}")
print(f"Min={ppq.min()}, Max={ppq.max()}")

Raw data: 2,621,288 examples, 1,814,924 products
Label distribution:
esci_label
E    1708158
S     574313
I     263165
C      75652
Name: count, dtype: int64

After filtering: 1,247,558 rows
Unique queries: 97,344
Unique products: 904,348

Products per query: mean=12.8, median=14
Min=1, Max=146


In [4]:
# ============================================================================
# CELL 2: Sampling Strategy
# ============================================================================
"""
Decision: Sample 50 queries first, then take 500 rows from those queries
Reasoning: Ensures diverse queries while meeting row count requirement
"""

np.random.seed(42)

# Sample queries
query_ids = filtered['query_id'].unique()
sampled_qids = np.random.choice(query_ids, size=50, replace=False)

# Get all rows for those queries
subset = filtered[filtered['query_id'].isin(sampled_qids)]
print(f"Rows with sampled queries: {len(subset):,}")

# Take 500 rows
sample = subset.sample(n=min(500, len(subset)), random_state=42)

print(f"\nFinal sample:")
print(f"  Rows: {len(sample)}")
print(f"  Queries: {sample['query_id'].nunique()}")
print(f"  Products: {sample['product_id'].nunique()}")

# Quick peek at query diversity
print("\nSample queries:")
for q in sample['query'].drop_duplicates().head(5):
    print(f"  - {q}")

sample.to_csv("sample_dataset.csv", index=False)
print("\n Saved to sample_dataset.csv")

Rows with sampled queries: 692

Final sample:
  Rows: 500
  Queries: 49
  Products: 499

Sample queries:
  - pickled white wood stain
  - women's birthday gifts
  - iblason phone case
  - magnolia market
  - lightweight messenger bag for women

 Saved to sample_dataset.csv


In [3]:
# ============================================================================
# CELL 3: Product Text Preparation
# ============================================================================
"""
Key decision: Concatenate all product fields for richer representations
- Title (most important)
- Brand, color (important for filtering)
- Bullets, description (detailed info)

Based on the results in Production, I would also do some weighted sum -
giving more importance to title, then brand, then product_bullet_point,
then product_description.
"""

# Merge sample with products
sample_keys = sample[['product_id', 'product_locale']].drop_duplicates()
prod = sample_keys.merge(
    products[['product_id', 'product_locale', 'product_title', 'product_brand', 
              'product_bullet_point', 'product_description']], 
    on=['product_id', 'product_locale'], 
    how='left'
)

# Build combined text
text_cols = ['product_title', 'product_brand', 'product_bullet_point', 'product_description']
prod['product_text'] = prod[text_cols].fillna('').agg(' '.join, axis=1)
prod['product_text'] = prod['product_text'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Check quality
prod = prod[prod['product_text'].str.len() > 0].copy()
print(f"Products with text: {len(prod)}")
print(f"Avg text length: {prod['product_text'].str.len().mean():.0f} chars")

# Show example
print("\nExample product text:")
print(prod.iloc[0]['product_text'][:200] + "...")



Products with text: 499
Avg text length: 1363 chars

Example product text:
Minwax Wood Finish 227614444, Classic Gray Stain, Half Pint Minwax RICH EVEN COLOR – Minwax Wood Finish is a deep penetrating, oil-based wood stain that provides beautiful color and enhances the natur...


In [4]:
# ============================================================================
# CELL 4: Baseline - Dense Embeddings Only
# ============================================================================
"""
Approach 1: Pure semantic search with sentence transformers
Model choice: all-MiniLM-L6-v2
- Fast (384 dim)
- Standard baseline
Vector Database Choice: Using FAISS in-memory vector store since the sample data is only 500 rows.
"""

from sentence_transformers import SentenceTransformer
import faiss

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print(f"Model loaded: {model.get_sentence_embedding_dimension()} dimensions")

# Encode products
prod_texts = prod['product_text'].tolist()
prod_ids = (prod['product_id'].astype(str) + '||' + prod['product_locale'].astype(str)).tolist()

print("Encoding products...")
embs = model.encode(prod_texts, batch_size=128, show_progress_bar=True, 
                   normalize_embeddings=True) #processes 128 products at a time

# Build FAISS index
index = faiss.IndexFlatIP(embs.shape[1])
index.add(embs.astype('float32'))
print(f"Index built: {index.ntotal} vectors")

C:\Users\Jahnavi Gajula\.conda\envs\ml-dml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded: 384 dimensions
Encoding products...


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 4/4 [00:23<00:00,  5.77s/it]

Index built: 499 vectors


In [5]:
# ============================================================================
# CELL 5: Evaluation Setup
# ============================================================================
"""
Ground truth: All sample rows are relevant (esci_label='E')
Metrics: HITS@1, HITS@5, HITS@10, MRR
"""

# Build ground truth dict
sample['_pid_key'] = (sample['product_id'].astype(str) + '||' + 
                      sample['product_locale'].astype(str))
gt = sample.groupby('query_id').agg({
    'query': 'first',
    '_pid_key': list
}).to_dict('index')

print(f"Ground truth: {len(gt)} queries")
print(f"Avg relevant per query: {sample.groupby('query_id').size().mean():.1f}")

def hits_at_k(ranks, k):
    return 1.0 if any(r < k for r in ranks) else 0.0

def mrr(ranks):
    return 0.0 if not ranks else 1.0 / (min(ranks) + 1)

def eval_ranking(ranked_pids, relevant_pids):
    ranks = [i for i, pid in enumerate(ranked_pids) if pid in relevant_pids]
    return {
        'hits@1': hits_at_k(ranks, 1),
        'hits@5': hits_at_k(ranks, 5),
        'hits@10': hits_at_k(ranks, 10),
        'mrr': mrr(ranks)
    }



Ground truth: 49 queries
Avg relevant per query: 10.2


In [6]:
# ============================================================================
# CELL 6: Baseline Results
# ============================================================================
"""
Test pure embeddings approach
"""

from tqdm import tqdm

results_baseline = []

for qid, data in tqdm(gt.items(), desc="Evaluating baseline"):
    q_emb = model.encode([data['query']], normalize_embeddings=True)
    D, I = index.search(q_emb.astype('float32'), k=10)
    
    ranked = [prod_ids[i] for i in I[0]]
    metrics = eval_ranking(ranked, set(data['_pid_key']))
    
    results_baseline.append({
        'query_id': qid,
        'query': data['query'],
        **metrics
    })

df_baseline = pd.DataFrame(results_baseline)

print("\n=== BASELINE: all-MiniLM-L6-v2 ===")
print(f"HITS@1:  {df_baseline['hits@1'].mean():.3f}")
print(f"HITS@5:  {df_baseline['hits@5'].mean():.3f}")
print(f"HITS@10: {df_baseline['hits@10'].mean():.3f}")
print(f"MRR:     {df_baseline['mrr'].mean():.3f}")

# Find problem cases
worst = df_baseline.nsmallest(3, 'mrr')
print(f"\nWorst queries (MRR):")
for _, row in worst.iterrows():
    print(f"  {row['mrr']:.2f} - '{row['query']}'")

Evaluating baseline: 100%|█████████████████████████████████████████████████████████████| 49/49 [00:01<00:00, 33.05it/s]


=== BASELINE: all-MiniLM-L6-v2 ===
HITS@1:  0.980
HITS@5:  1.000
HITS@10: 1.000
MRR:     0.990

Worst queries (MRR):
  0.50 - 'travel size fragrance spray'
  1.00 - '20 inch storage bin'
  1.00 - 'alpha chi omega'


In [7]:
# ============================================================================
# CELL 7: Alternative Embedding Model - E5-Small-v2
# ============================================================================
"""
Experiment: Try E5-small-v2 (query-focused training)
- Same 384 dimensions as MiniLM
- Trained specifically on query-document matching
- Instruction-based (requires 'query:' prefix)

Decision point: Does query-specific training help?
"""

model_e5 = SentenceTransformer('intfloat/e5-small-v2')
print(f"E5 model loaded: {model_e5.get_sentence_embedding_dimension()} dimensions")

# E5 requires instruction prefixes
print("\nEncoding products with E5...")
prod_texts_e5 = ["passage: " + t for t in prod_texts]
embs_e5 = model_e5.encode(prod_texts_e5, batch_size=128, show_progress_bar=True,
                          normalize_embeddings=True)

# Build E5 index
index_e5 = faiss.IndexFlatIP(embs_e5.shape[1])
index_e5.add(embs_e5.astype('float32'))
print(f"E5 index built: {index_e5.ntotal} vectors")

# Evaluate E5
results_e5 = []
for qid, data in tqdm(gt.items(), desc="Evaluating E5"):
    q_text_e5 = "query: " + data['query']
    q_emb = model_e5.encode([q_text_e5], normalize_embeddings=True)
    D, I = index_e5.search(q_emb.astype('float32'), k=10)
    
    ranked = [prod_ids[i] for i in I[0]]
    metrics = eval_ranking(ranked, set(data['_pid_key']))
    
    results_e5.append({
        'query_id': qid,
        'query': data['query'],
        **metrics
    })

df_e5 = pd.DataFrame(results_e5)

E5 model loaded: 384 dimensions

Encoding products with E5...


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 4/4 [01:29<00:00, 22.30s/it]


E5 index built: 499 vectors


Evaluating E5: 100%|███████████████████████████████████████████████████████████████████| 49/49 [00:03<00:00, 14.64it/s]


In [8]:
# Analysis of E5- small vs BERT mini
print("\n=== E5-small-v2 vs MiniLM ===")
print(f"{'Metric':<10} {'MiniLM':>10} {'E5':>10} {'Diff':>8}")
print("-" * 42)
for metric in ['hits@1', 'hits@5', 'hits@10', 'mrr']:
    mini = df_baseline[metric].mean()
    e5 = df_e5[metric].mean()
    print(f"{metric.upper():<10} {mini:>10.3f} {e5:>10.3f} {e5-mini:>+8.3f}")



=== E5-small-v2 vs MiniLM ===
Metric         MiniLM         E5     Diff
------------------------------------------
HITS@1          0.980      1.000   +0.020
HITS@5          1.000      1.000   +0.000
HITS@10         1.000      1.000   +0.000
MRR             0.990      1.000   +0.010


#### E5 is performing better. However, since this is a POC on a small dataset, the same results may not hold for larger datasets or production pipelines. In such cases, we would likely need to incorporate reranking or other mechanisms. For now, I would like to continue experimenting with different ranking approaches. If I observed similar results in a production setting, I would stop here and proceed with E5.

In [9]:
# Deep dive: How did E5 fix the problematic query?
problem_qid = 105041  # "travel size fragrance spray"
if problem_qid in gt:
    problem_query = gt[problem_qid]['query']
    relevant_pids = set(gt[problem_qid]['_pid_key'])
    
    print(f"\nQuery {problem_qid} '{problem_query}'")
    print(f"Relevant products: {len(relevant_pids)}")
    
    # MiniLM results
    q_emb_mini = model.encode([problem_query], normalize_embeddings=True)
    D_mini, I_mini = index.search(q_emb_mini.astype('float32'), k=10)
    ranked_mini = [prod_ids[i] for i in I_mini[0]]
    
    # E5 results
    q_emb_e5 = model_e5.encode(["query: " + problem_query], normalize_embeddings=True)
    D_e5, I_e5 = index_e5.search(q_emb_e5.astype('float32'), k=10)
    ranked_e5 = [prod_ids[i] for i in I_e5[0]]
    
    print("\nMiniLM Top-3:")
    for i, pid in enumerate(ranked_mini[:3], 1):
        mark = "[HIT]" if pid in relevant_pids else "[MISS]"
        idx = prod_ids.index(pid)
        title = prod.iloc[idx]['product_title'][:55]
        print(f"  {i}. {mark} {title}")
    
    print("\nE5 Top-3:")
    for i, pid in enumerate(ranked_e5[:3], 1):
        mark = "[HIT]" if pid in relevant_pids else "[MISS]"
        idx = prod_ids.index(pid)
        title = prod.iloc[idx]['product_title'][:55]
        print(f"  {i}. {mark} {title}")
    
    # Compute metrics for this query
    mini_metrics = eval_ranking(ranked_mini, relevant_pids)
    e5_metrics = eval_ranking(ranked_e5, relevant_pids)


 Query 105041 'travel size fragrance spray'
Relevant products: 10

MiniLM Top-3:
  1. [MISS] Pink by Victoria's Secret Eau De Parfum Spray for Women
  2. [HIT] Victoria's Secret Bombshell Body Mist 2.5oz Travel Size
  3. [HIT] Guerlain Mon Guerlain Eau De Parfum Mini Spray for Wome

E5 Top-3:
  1. [HIT] Victoria's Secret Bombshell Body Mist 2.5oz Travel Size
  2. [HIT] Inis the Energy of the Sea Travel Cologne Spray, 0.5 Fl
  3. [HIT] Tocca Travel Fragrance Spray - Cleopatra - 0.68 oz


In [10]:
# Since E5 is trained on large corpus of query-document this helped, but still want to explore further
# For POC: Continue with MiniLM to show full experimental journey
# In production: Would use E5
model_final = model
index_final = index
df_best = df_baseline
prefix_query = ""
prefix_doc = ""
model_name = "all-MiniLM-L6-v2"

In [11]:
# ============================================================================
# CELL 8: Root Cause Analysis (MiniLM Baseline)
# ============================================================================
"""
Even though E5 fixed the problem, let's analyze WHY MiniLM failed
This helps understand if hybrid approach would be even more robust

Pure embeddings miss exact keyword matches
Example: "travel size fragrance spray" - needs exact "travel size" match
"""

# Analyze the problematic MiniLM query we saw earlier
qid = 105041  # travel size fragrance spray
qtext = gt[qid]['query']

print(f"Analyzing MiniLM failure: '{qtext}'")

# MiniLM results
q_emb = model.encode([qtext], normalize_embeddings=True)
D, I = index.search(q_emb.astype('float32'), k=10)
ranked = [prod_ids[i] for i in I[0]]

relevant = set(gt[qid]['_pid_key'])
print(f"\nRelevant items: {len(relevant)}")
print("\nMiniLM Top 10:")
for i, pid in enumerate(ranked[:10]):
    is_rel = "[HIT]" if pid in relevant else "[MISS]"
    idx = prod_ids.index(pid)
    title = prod.iloc[idx]['product_title'][:60]
    print(f"{i+1:2}. {is_rel} {title}")



Analyzing MiniLM failure: 'travel size fragrance spray'

Relevant items: 10

MiniLM Top 10:
 1. [MISS] Pink by Victoria's Secret Eau De Parfum Spray for Women, 2.5
 2. [HIT] Victoria's Secret Bombshell Body Mist 2.5oz Travel Size
 3. [HIT] Guerlain Mon Guerlain Eau De Parfum Mini Spray for Women, 0.
 4. [HIT] Tocca Travel Fragrance Spray - Cleopatra - 0.68 oz
 5. [HIT] Emeraude Exclamation Cologne Body Spray by Emeraude 2.5 Flui
 6. [HIT] Viktor & Rolf Flowerbomb 0.68 oz Eau de Parfum Spray Fragran
 7. [HIT] Inis the Energy of the Sea Travel Cologne Spray, 0.5 Fluid O
 8. [MISS] Victoria's Secret Love Pink Eau de Parfum Spray, 1.7 Ounce
 9. [MISS] Victoria's Secret Pink Pink Gold Eau De Parfum 1 Ounce (30 M
10. [HIT] Tocca Travel Spray Eau de Parfum, 20ml (Stella)


Root Cause: Generic 'spray' and 'fragrance' match semantically but missing Exact 'travel size' keyword signal



In [12]:
# ============================================================================
# CELL 9: Experiment - Cross-Encoder Reranking
# ============================================================================
"""
Attempted Solution 1: Add cross-encoder reranker
Goal: Use more powerful model to rerank top-K candidates
One thing to note here is added latency might not be worth marginal gains
Cross encoder also provides the relevance scores - for each query document pair
"""

from sentence_transformers import CrossEncoder

reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512)
print("Cross-encoder loaded")

def search_with_rerank(qtext, k=50):
    """Two-stage: FAISS retrieve + CE rerank"""
    # Stage 1: FAISS
    q_emb = model_final.encode([prefix_query + qtext], normalize_embeddings=True)
    _, I = index_final.search(q_emb.astype('float32'), k=k)
    candidates = [prod_ids[i] for i in I[0]]
    
    # Stage 2: Cross-encoder
    pairs = [(qtext, prod.iloc[prod_ids.index(pid)]['product_text']) 
             for pid in candidates]
    scores = reranker.predict(pairs)
    
    # Sort by CE score
    return [candidates[i] for i in scores.argsort()[::-1]]

# Test on problem query
rerank_result = search_with_rerank(qtext)
print(f"\nReranked results for: '{qtext}'")
print("Top 10:")
for i, pid in enumerate(rerank_result[:10]):
    is_rel = "[HIT]" if pid in relevant else "[MISS]"
    idx = prod_ids.index(pid)
    title = prod.iloc[idx]['product_title'][:60]
    print(f"{i+1:2}. {is_rel} {title}")

# Quick eval
results_rerank = []
for qid, data in tqdm(gt.items(), desc="Evaluating rerank"):
    ranked = search_with_rerank(data['query'])
    metrics = eval_ranking(ranked, set(data['_pid_key']))
    results_rerank.append({'query_id': qid, **metrics})

df_rerank = pd.DataFrame(results_rerank)

print(f"\n=== Cross-Encoder Reranking ===")
print(f"HITS@1: {df_rerank['hits@1'].mean():.3f} (was {df_best['hits@1'].mean():.3f})")
print(f"MRR:    {df_rerank['mrr'].mean():.3f} (was {df_best['mrr'].mean():.3f})")


Cross-encoder loaded

Reranked results for: 'travel size fragrance spray'
Top 10:
 1. [HIT] Tocca Travel Fragrance Spray - Cleopatra - 0.68 oz
 2. [HIT] Inis the Energy of the Sea Travel Cologne Spray, 0.5 Fluid O
 3. [HIT] Victoria's Secret Bombshell Body Mist 2.5oz Travel Size
 4. [HIT] Tocca Travel Spray Eau de Parfum, 20ml (Stella)
 5. [HIT] Guerlain Mon Guerlain Eau De Parfum Mini Spray for Women, 0.
 6. [MISS] Pink by Victoria's Secret Eau De Parfum Spray for Women, 2.5
 7. [HIT] Viktor & Rolf Flowerbomb 0.68 oz Eau de Parfum Spray Fragran
 8. [HIT] Emeraude Exclamation Cologne Body Spray by Emeraude 2.5 Flui
 9. [HIT] Body Fantasies Signature Fragrance Body Spray, Japanese Cher
10. [MISS] Victoria's Secret Love Pink Eau de Parfum Spray, 1.7 Ounce


Evaluating rerank: 100%|███████████████████████████████████████████████████████████████| 49/49 [05:02<00:00,  6.17s/it]


=== Cross-Encoder Reranking ===
HITS@1: 1.000 (was 0.980)
MRR:    1.000 (was 0.990)


In [13]:
#Analysis
print(f"   Improvement: {(df_rerank['mrr'].mean() - df_best['mrr'].mean())*100:.1f}% MRR gain")

   Improvement: 1.0% MRR gain


This reranking model has cause additional latency per query, Let' see if we can Try lighter hybrid approach first

In [14]:
# ============================================================================
# CELL 10: Hybrid Approach - Embeddings + BM25
# ============================================================================
"""
Combine embeddings (semantic) + BM25 (strong lexical baseline)
Use Reciprocal Rank Fusion (RRF)
Reason: BM25 is a widely used, production-friendly sparse retriever
"""


import re
from typing import List
from rank_bm25 import BM25Okapi

def _simple_tokenize(text: str) -> List[str]:
    # lightweight, fast tokenizer good enough for BM25
    return re.findall(r"[a-z0-9]+", text.lower())

# Build BM25 index (ensure prod_texts aligns with prod_ids)
corpus_tokens = [_simple_tokenize(t) for t in prod_texts]
bm25 = BM25Okapi(corpus_tokens, k1=1.5, b=0.75)

def search_hybrid(qtext, k=100, rrf_k=60):
    """Hybrid: embeddings + BM25 with RRF fusion."""
    # Dense retrieval via FAISS
    q_emb = model_final.encode([prefix_query + qtext], normalize_embeddings=True)
    _, faiss_idx = index_final.search(q_emb.astype('float32'), k=k)
    faiss_pids = [prod_ids[i] for i in faiss_idx[0]]

    # Sparse retrieval via BM25
    q_tokens = _simple_tokenize(qtext)
    bm25_scores = bm25.get_scores(q_tokens)  # array aligned to corpus/prod_ids
    bm25_idx = bm25_scores.argsort()[::-1][:k]
    bm25_pids = [prod_ids[i] for i in bm25_idx]

    # RRF fusion (higher is better)
    rrf = {}
    for rank, pid in enumerate(faiss_pids):
        rrf[pid] = rrf.get(pid, 0.0) + 1.0 / (rrf_k + rank + 1)
    for rank, pid in enumerate(bm25_pids):
        rrf[pid] = rrf.get(pid, 0.0) + 1.0 / (rrf_k + rank + 1)

    # Sort by fused score
    fused = sorted(rrf.items(), key=lambda kv: -kv[1])
    return [pid for pid, _ in fused]

# Test on the same problem query
hybrid_result = search_hybrid(qtext)
print(f"\nHybrid (Embeddings + BM25) results for: '{qtext}'")
print("Top 10:")
for i, pid in enumerate(hybrid_result[:10]):
    is_rel = "[HIT]" if pid in relevant else "[MISS]"
    idx = prod_ids.index(pid)
    title = prod.iloc[idx]['product_title'][:60]
    print(f"{i+1:2}. {is_rel} {title}")



Hybrid (Embeddings + BM25) results for: 'travel size fragrance spray'
Top 10:
 1. [HIT] Tocca Travel Fragrance Spray - Cleopatra - 0.68 oz
 2. [HIT] Guerlain Mon Guerlain Eau De Parfum Mini Spray for Women, 0.
 3. [MISS] Pink by Victoria's Secret Eau De Parfum Spray for Women, 2.5
 4. [HIT] Victoria's Secret Bombshell Body Mist 2.5oz Travel Size
 5. [HIT] Inis the Energy of the Sea Travel Cologne Spray, 0.5 Fluid O
 6. [HIT] Emeraude Exclamation Cologne Body Spray by Emeraude 2.5 Flui
 7. [HIT] Tocca Travel Spray Eau de Parfum, 20ml (Stella)
 8. [HIT] Viktor & Rolf Flowerbomb 0.68 oz Eau de Parfum Spray Fragran
 9. [MISS] Victoria's Secret Love Pink Eau de Parfum Spray, 1.7 Ounce
10. [HIT] Body Fantasies Signature Fragrance Body Spray, Japanese Cher


In this case, I observed that “MISS” appears in 3rd place with the Hybrid setup (Embeddings + BM25 ) versus 6th place with the cross-encoder. This indicates that query quality is affected slightly. In a production setting, I would validate this more rigorously using ranking metrics such as NDCG (Normalized Discounted Cumulative Gain).

That said, the final choice depends on the trade-off between speed and accuracy. For this use case, I believe speed is more critical, so I would lean toward adopting lightweight hybrid embeddings that provide faster inference while maintaining acceptable ranking quality.

In [15]:
# ============================================================================
# CELL 11: Hybrid Evaluation & Comparison
# ============================================================================
"""
Final comparison: Baseline vs Reranker vs Hybrid
"""

results_hybrid = []
for qid, data in tqdm(gt.items(), desc="Evaluating hybrid"):
    ranked = search_hybrid(data['query'])
    metrics = eval_ranking(ranked, set(data['_pid_key']))
    results_hybrid.append({'query_id': qid, 'query': data['query'], **metrics})

df_hybrid = pd.DataFrame(results_hybrid)

print("\n=== FINAL COMPARISON ===")
print(f"{'Metric':<10} {'Baseline':>10} {'Reranker':>10} {'Hybrid':>10}")
print("-" * 54)
for metric in ['hits@1', 'hits@5', 'hits@10', 'mrr']:
    base = df_best[metric].mean()
    rerank = df_rerank[metric].mean()
    hyb = df_hybrid[metric].mean()
    print(f"{metric.upper():<10} {base:>10.3f} {rerank:>10.3f} {hyb:>10.3f}")

Evaluating hybrid: 100%|███████████████████████████████████████████████████████████████| 49/49 [00:02<00:00, 24.22it/s]


=== FINAL COMPARISON ===
Metric       Baseline   Reranker     Hybrid
------------------------------------------------------
HITS@1          0.980      1.000      1.000
HITS@5          1.000      1.000      1.000
HITS@10         1.000      1.000      1.000
MRR             0.990      1.000      1.000


In [16]:
#============================================================================
# CELL 12: Production Insights & Next Steps
# ============================================================================
"""
Key Learnings & Production Recommendations
"""

import json
import os

# Save all results
os.makedirs('artifacts', exist_ok=True)
df_baseline.to_csv('artifacts/results_baseline.csv', index=False)
df_e5.to_csv('artifacts/results_e5.csv', index=False)
df_rerank.to_csv('artifacts/results_rerank.csv', index=False)
df_hybrid.to_csv('artifacts/results_hybrid.csv', index=False)

metrics = {
    'baseline_minilm': {
        'model': 'all-MiniLM-L6-v2',
        'mrr': float(df_baseline['mrr'].mean()),
        'hits@1': float(df_baseline['hits@1'].mean())
    },
    'baseline_e5': {
        'model': 'e5-small-v2',
        'mrr': float(df_e5['mrr'].mean()),
        'hits@1': float(df_e5['hits@1'].mean()),
        'note': 'Already perfect - query-specific training works'
    },
    'reranker_minilm': {
        'approach': 'MiniLM + cross-encoder rerank',
        'mrr': float(df_rerank['mrr'].mean()),
        'hits@1': float(df_rerank['hits@1'].mean()),
        'latency_overhead': '~50-100ms'
    },
    'hybrid_minilm': {
        'approach': 'MiniLM + BM25 + RRF',
        'mrr': float(df_hybrid['mrr'].mean()),
        'hits@1': float(df_hybrid['hits@1'].mean()),
        'latency_overhead': '~5ms'
    }
}

In [17]:
with open('artifacts/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\n=== KEY FINDINGS ===")
print("\n1. Model Selection:")
print(f"   MiniLM: {df_baseline['mrr'].mean():.3f} MRR, {df_baseline['hits@1'].mean():.3f} HITS@1")
print(f"   E5:     {df_e5['mrr'].mean():.3f} MRR, {df_e5['hits@1'].mean():.3f} HITS@1")
print(f"   → E5's query-document training solved the baseline problem")
print(f"   → Diff MRR = +{(df_e5['mrr'].mean() - df_baseline['mrr'].mean()):.3f}")

print("\n2. Retrieval Enhancement (explored on MiniLM):")
print(f"   Pure embeddings:     {df_baseline['hits@1'].mean():.3f} HITS@1")
print(f"   + Cross-encoder:     {df_rerank['hits@1'].mean():.3f} HITS@1")
print(f"   + BM25 hybrid:     {df_hybrid['hits@1'].mean():.3f} HITS@1")


=== KEY FINDINGS ===

1. Model Selection:
   MiniLM: 0.990 MRR, 0.980 HITS@1
   E5:     1.000 MRR, 1.000 HITS@1
   → E5's query-document training solved the baseline problem
   → Diff MRR = +0.010

2. Retrieval Enhancement (explored on MiniLM):
   Pure embeddings:     0.980 HITS@1
   + Cross-encoder:     1.000 HITS@1
   + BM25 hybrid:     1.000 HITS@1


At the end, I would add logging to track latency and response times in real use. This would show me how much delay affects business outcomes compared to accuracy gains. If needed, I would run statistical tests to check if the differences are meaningful and then make decisions based on that.

### NEXT STEPS (Production Considerations):

Use OpenSearch with HNSW instead of just in-memory Faiss, since it’s production-ready, scalable, and widely adopted.

Try out ColBERT v2 or similar late-interaction models, which are commonly used in real-world search deployments.

Add query normalization (lowercasing, removing stopwords, spelling fixes, and dictionary-based mappings (abbreviations → full form)) to reduce noise and improve recall.

Evaluate with per-query recall and error analysis to understand exactly where the system misses and why.

Experiment with a weighted fusion of product fields (title > brand > bullet_point > description) since title usually carries the strongest signal.

Continuously run production assessments to see which design choices actually improve user experience and business metrics.